<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#ABC-model-of-metabolism" data-toc-modified-id="ABC-model-of-metabolism-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>ABC model of metabolism</a></span><ul class="toc-item"><li><span><a href="#Implementation-of-the-ABC-model-using-Cobrapy" data-toc-modified-id="Implementation-of-the-ABC-model-using-Cobrapy-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Implementation of the ABC model using Cobrapy</a></span></li></ul></li><li><span><a href="#Steady-state-analysis-of-the-ABC-model" data-toc-modified-id="Steady-state-analysis-of-the-ABC-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Steady state analysis of the ABC model</a></span></li><li><span><a href="#Cell-factory-design-questions-for-the-ABC-model" data-toc-modified-id="Cell-factory-design-questions-for-the-ABC-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Cell factory design questions for the ABC model</a></span><ul class="toc-item"><li><span><a href="#What-is-the-maximum-achievable-growth-rate?" data-toc-modified-id="What-is-the-maximum-achievable-growth-rate?-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>What is the maximum achievable growth rate?</a></span><ul class="toc-item"><li><span><a href="#Cobrapy-implementation-of-the-optimization-problem" data-toc-modified-id="Cobrapy-implementation-of-the-optimization-problem-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Cobrapy implementation of the optimization problem</a></span></li><li><span><a href="#We-can-also-visualize-the-fluxes-on-the-network" data-toc-modified-id="We-can-also-visualize-the-fluxes-on-the-network-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>We can also visualize the fluxes on the network</a></span></li></ul></li><li><span><a href="#What-is-the-maximum-bioproduct-yield?" data-toc-modified-id="What-is-the-maximum-bioproduct-yield?-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>What is the maximum bioproduct yield?</a></span></li><li><span><a href="#How-can-we-shift-environmental-conditions-to-optimize-bioproduct-yield?" data-toc-modified-id="How-can-we-shift-environmental-conditions-to-optimize-bioproduct-yield?-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>How can we shift environmental conditions to optimize bioproduct yield?</a></span></li><li><span><a href="#How-can-we-use-gene-knockouts-to-optimize-bioproduct-yield?" data-toc-modified-id="How-can-we-use-gene-knockouts-to-optimize-bioproduct-yield?-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>How can we use gene knockouts to optimize bioproduct yield?</a></span></li><li><span><a href="#How-can-we-balance-the-tradeoff-between-growth-rate-and-bioproduct-yield?" data-toc-modified-id="How-can-we-balance-the-tradeoff-between-growth-rate-and-bioproduct-yield?-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>How can we balance the tradeoff between growth rate and bioproduct yield?</a></span></li></ul></li></ul></div>

# ABC model of metabolism

Consider a cell, where we track all the metabolites going into and out of the cell through reactions.
![ABC_network.png](ABC/Metabolic-network.JPG)


The chemical equations for the ABC model are:

$$
\begin{array}{lrcl}
 R_1: & & \overset{v_1}{\rightarrow} & A \\
 R_2: & A & \overset{v_2}{\rightarrow} & B \\
 R_3: & A & \overset{v_3}{\rightarrow} & C \\
 R_4: & B + E & \overset{v_4}{\rightarrow}& 2D\\
 R_5:  & &\overset{v_5}{\rightarrow} & E \\
 R_6: & 2B & \overset{v_6}{\rightarrow} & C + F \\
 R_7: & C & \overset{v_7}{\rightarrow} & D \\
 R_8: & D & \overset{v_8}{\rightarrow} &  \\
 R_9: & F & \overset{v_9}{\rightarrow} &  \\
\end{array}
$$

where $R_j$ are the reaction equations, the $v_j$ are the reaction rates, or fluxes, and the arrow direction indicates the reaction is irreversible.
These chemical equations can also be represented as a Stoichiometric matrix $S$:
 
$$
S = \left[ {\begin{array}{cccccccccc}
  & R_1 & R_2 & R_3 & R_4 & R_5 & R_6 & R_7 & R_8 & R_9 \\
A & 1   & -1  & -1  & 0   & 0   & 0   & 0   & 0   & 0   \\
B & 0   &  1  & 0   & -1  & 0   & -2  & 0   & 0   & 0   \\
C & 0   &  0  & 1   & 0   & 0   & 1   & -1  & 0   & 0   \\
D & 0   &  0  & 0   & 2   & 0   & 0   &  1  & -1  & 0   \\
E & 0   &  0  & 0   & -1   & 1   & 0   &  0  & 0  & 0   \\
F & 0   &  0  & 0   &  0   & 0   & 1   &  0  & 0  & -1   \\
\end{array}}\right]
$$

where each column corresponds to a reaction, each row corresponds to a metabolite, and the element corresponds to the stoichiometry of each metabolite in that reaction. The number is negative if the metabolite is a reactant of the reaction, positive if it is a product of the reaction, and zero, otherwise.

## Implementation of the ABC model using Cobrapy

In [ ]:
import cobra
abc_model = cobra.Model('ABC_model')

A  = cobra.Metabolite('A',compartment='c')
B  = cobra.Metabolite('B',compartment='c')
C  = cobra.Metabolite('C',compartment='c')
D  = cobra.Metabolite('D',compartment='c')
E  = cobra.Metabolite('E',compartment='c')
F  = cobra.Metabolite('F',compartment='c')

abc_model.add_metabolites([A,B,C,D,E,F])

R_1 = cobra.Reaction('R_1')
R_2 = cobra.Reaction('R_2')
R_3 = cobra.Reaction('R_3')
R_4 = cobra.Reaction('R_4')
R_5 = cobra.Reaction('R_5')
R_6 = cobra.Reaction('R_6')
R_7 = cobra.Reaction('R_7')
R_8 = cobra.Reaction('R_8')
R_9 = cobra.Reaction('R_9')

abc_model.add_reactions([R_1, R_2, R_3, R_4, R_5, R_6, R_7, R_8, R_9])

R_1.build_reaction_from_string('--> A')
R_2.build_reaction_from_string('A --> B')
R_3.build_reaction_from_string('A --> C')
R_4.build_reaction_from_string('B + E --> 2 D')
R_5.build_reaction_from_string('--> E')
R_6.build_reaction_from_string('2 B --> C + F')
R_7.build_reaction_from_string('C --> D')
R_8.build_reaction_from_string('D -->')
R_9.build_reaction_from_string('F -->')

cobra.io.save_json_model(abc_model, 'ABC/abc_model.json')
cobra.util.array.create_stoichiometric_matrix(abc_model, 
                                              array_type='DataFrame').astype(int)

# Steady state analysis of the ABC model

![steady state pools of water](ABC/Pamukkale.jpg)

Like the terraced pools of water in the geothermal hot springs of [Pamukkale, Turkey](https://rustytraveltrunk.com/pamukkale/), when the metabolic network is in steady state, the concentrations of the internal metabolites do not change. Therefore
$$ \frac{d\vec{c}}{dt} = S\cdot\vec{v} = 0$$
where $\frac{d\vec{c}}{dt}$ represents the change in metabolite concentrations with respect to time, and $\vec{v}$ are the  reaction rates (also known as fluxes), and $S$ is the stoichiometric matrix.


$$
\left[ {\begin{array}{c}
\frac{dA}{dt} \\ \frac{dB}{dt} \\ \frac{dC}{dt} \\ \frac{dD}{dt} \\ \frac{dE}{dt} \\ \frac{dF}{dt} \\
\end{array}}\right] = 
S\cdot\vec{v} = \left[ {\begin{array}{cccccccccc}
  & R_1 & R_2 & R_3 & R_4 & R_5 & R_6 & R_7 & R_8 & R_9 \\
A & 1   & -1  & -1  & 0   & 0   & 0   & 0   & 0   & 0   \\
B & 0   &  1  & 0   & -1  & 0   & -2  & 0   & 0   & 0   \\
C & 0   &  0  & 1   & 0   & 0   & 1   & -1  & 0   & 0   \\
D & 0   &  0  & 0   & 2   & 0   & 0   &  1  & -1  & 0   \\
E & 0   &  0  & 0   & -1   & 1   & 0   &  0  & 0  & 0   \\
F & 0   &  0  & 0   &  0   & 0   & 1   &  0  & 0  & -1   \\
\end{array}}\right]\cdot\left[ {\begin{array}{c}
v_1 \\ v_2 \\ v_3 \\ v_4 \\ v_5 \\ v_6 \\ v_7 \\ v_8 \\ v_9 \\
\end{array}}\right] = \left[ {\begin{array}{c}
v_1 - v_2 - v_3 \\ v_2 - v_4 - 2v_6 \\ v_3 + v_6 - v_7 \\ 2v_4 + v_7 - v_8 \\ -v_4 + v_5 \\ v_6 - v_9 \\
\end{array}}\right] = 0
$$



![ELMO](ABC/ELMO.JPG)

For the ABC model, all feasible steady-state flux distributions can be decomposed into non-negative combinations of just 3 elementary modes. Although elementary modes are useful conceptual framework for analyzing small networks, they are not practical for genome-scale network analysis because the number of elementary modes increases exponentially with the size of the network.  Nevertheless, for this ABC model keeping in mind these three elementary modes will be helpful when solving the cell factory design problems below.


# Cell factory design questions for the ABC model

1. What is the maximum achievable growth rate?
2. What is the maximum bioproduct yield?
3. How can we alter experimental conditions to achieve maximum bioproduct yield?
4. How do we use gene knockouts to achieve maximum bioproduct yield?
5. How can we balance the tradeoff between growth rate and bioproduct yield?


## What is the maximum achievable growth rate?

Suppose that the uptake rate of $A$ is limited to $10 \frac{mmol}{hour}$, and $D$ is biomass. What is the maximum growth rate achievable given the constraints of this network?  We can find the answer by solving the following optimization problem:
$$\begin{array}{ll}
    \underset{\vec{v}}{\mbox{maximize}}   & v_{8} \\
    \mbox{subject to}  & S\cdot \vec{v} = 0 \\
    & 0 \leq \vec{v}  \\
    & v_1 \leq 10 \\
\end{array}$$

### Cobrapy implementation of the optimization problem

In [ ]:
abc_model = cobra.io.load_json_model('ABC/abc_model.json')
abc_model.objective = R_8
abc_model.reactions.R_1.upper_bound = 10
abc_model.optimize()

### We can also visualize the fluxes on the network

In [ ]:
import escher
reaction_scale = [ { 'type': 'min',  'color': '#c8c8c8', 'size': 12 },
                   { 'type': 'mean', 'color': '#9696ff', 'size': 20 },
                   { 'type': 'max',  'color': '#ff0000', 'size': 25 } ]
escher.Builder(map_json='ABC/abc_map.json',
               model=abc_model,
               reaction_data=abc_model.optimize().fluxes.to_dict(),
               reaction_scale=reaction_scale
              ).display_in_notebook()




## What is the maximum bioproduct yield?

Since we are interested in microbial cell factories, let's think of $F$ as a high-value product.
What happens to the flux distribution and the growth rate if we maximize the production of $F$?

$$\begin{array}{ll}
    \underset{\vec{\bf v}}{\mbox{maximize}}   & v_9 \\
    \mbox{subject to}  & S\cdot v = 0 \\
    & 0 \leq v  \\
    & v_1 \leq 10 \\
\end{array}$$

In [ ]:
abc_model = cobra.io.load_json_model('ABC/abc_model.json')
abc_model.objective = R_9
abc_model.reactions.R_1.upper_bound = 10
R9_solution = abc_model.optimize()
display(R9_solution)
escher.Builder(map_json='ABC/abc_map.json',
               model=abc_model,
               reaction_data=R9_solution.fluxes.to_dict(),
               reaction_scale=reaction_scale
              ).display_in_notebook()

## How can we shift environmental conditions to optimize bioproduct yield?

We see from the previous exercise that if the cell wanted to produce the product, then there is a pathway that enables growth and bioproduction. But cells don't want to produce bioproducts, they just want to grow.  How can we persuade the cell to meet our objective?  In the ABC model, let's imagine that $E$ plays the role of oxygen in the metabolism of a facultative aerobe: having some makes the growth rate increase, but it is not strictly necessary for growth. What happens to the flux distribution if the cell tries to grow without $E$ (an-$E$-robically?)  Will this result in the desired pathway being utilized?

$$\begin{array}{ll}
    \underset{\vec{\bf v}}{\mbox{maximize}}   & v_{8} \\
    \mbox{subject to}  & S\cdot v = 0 \\
    & 0 \leq v  \\
    & v_1 \leq 10 \\
    & v_5 \leq 0 \\
\end{array}$$


In [ ]:
abc_model = cobra.io.load_json_model('ABC/ABC_model.json')
abc_model.objective = R_8
abc_model.reactions.R_1.upper_bound = 10
abc_model.reactions.R_5.upper_bound = 0
no_R5_solution = abc_model.optimize()
display(no_R5_solution)

escher.Builder(map_json='ABC/abc_map.json',
               model=abc_model,
               reaction_data=no_R5_solution.fluxes.to_dict(), 
               reaction_scale=reaction_scale,
              ).display_in_notebook()

## How can we use gene knockouts to optimize bioproduct yield?

As we discovered in the previous exercise, even with a small network, it isn't always obvious how to align the evolutionary objective of the cell with our engineering objective.  However, by knocking out the genes that enable alternate pathways, we can sculpt the metabolic network towards our objectives.

$$\begin{array}{ll}
    \underset{\vec{\bf v}}{\mbox{maximize}}   & v_{8} \\
    \mbox{subject to}  & S\cdot v = 0 \\
    & 0 \leq v  \\
    & v_1 \leq 10 \\
    & v_5 \leq 0 \\
    & v_3 \leq 0 \\
\end{array}$$


In [ ]:
abc_model = cobra.io.load_json_model('ABC/ABC_model.json')
abc_model.objective = R_8
abc_model.reactions.R_1.upper_bound = 10
abc_model.reactions.R_5.upper_bound = 0
abc_model.reactions.R_3.upper_bound = 0

display(abc_model.optimize())
escher.Builder(map_json='ABC/abc_map.json',
               model=abc_model,
               reaction_data=abc_model.optimize().fluxes.to_dict(), 
               reaction_scale=reaction_scale
              ).display_in_notebook()

## How can we balance the tradeoff between growth rate and bioproduct yield?

By knocking out $R_3$, we are able to maximize bioproduction, but growth rate is pretty low. We can balance the tradeoff between growth and bioproduction by modulating the uptake of $E$. 


$$\begin{array}{ll}
    \underset{\vec{\bf v}}{\mbox{maximize}}   & v_{8} \\
    \mbox{subject to}  & S\cdot v = 0 \\
    & 0 \leq v  \\
    & v_1 \leq 10 \\
    & v_5 \leq 5 \\
    & v_3 \leq 0 \\
\end{array}$$

In [ ]:
abc_model = cobra.io.load_json_model('ABC/ABC_model.json')
abc_model.objective = R_8
abc_model.reactions.R_1.upper_bound = 10
abc_model.reactions.R_5.upper_bound = 5
abc_model.reactions.R_3.upper_bound = 0
display(abc_model.optimize())
escher.Builder(map_json='ABC/abc_map.json',
               model=abc_model,
               reaction_data=abc_model.optimize().fluxes.to_dict(), 
               reaction_scale=reaction_scale
              ).display_in_notebook()

How much bioproduct ($F$) flux do we get per carbon source ($A$)?  How much does growth rate increase compared to the case where we are maximizing bioproduct yield?  Is this a reasonable tradeoff?